In [1]:
import numpy as np
from pytope import Polytope # not sure if it works for 3D
import picos
import scipy.optimize
import control
import itertools

from IntervalHull import qhull2D, minBoundingRect
from SE2Lie import *

/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
def solve_lmi(alpha, A, verbosity=0):
    
    prob = picos.Problem()
    P = picos.SymmetricVariable('P', (3, 3))
    P1 = P[:2, :]
    P2 = P[2, :]
    mu1 = picos.RealVariable('mu_1')
    mu2 = picos.RealVariable('mu_2')
    gam = mu1 + mu2
    for Ai in A:

        block_eq1 = picos.block([
            [Ai.T*P + P*Ai + alpha*P, P1.T, P2.T],
            [P1, -alpha*mu1*np.eye(2), 0],
            [P2, 0, -alpha*mu2]])
    
        prob.add_constraint(block_eq1 << 0) # dV < 0
    prob.add_constraint(P >> 1)
    prob.add_constraint(mu1 >> 0)
    prob.add_constraint(mu2 >> 0)
    prob.set_objective('min', mu1 + mu2)
    try:
        prob.solve(options={'verbosity': verbosity})
        cost = gam.value
    except Exception as e:
        print(e)
        cost = -1
    return {
        'cost': cost,
        'prob': prob,
        'mu1': mu1.value,
        'mu2': mu2.value,
        'P': np.round(np.array(P.value), 3),
        'alpha':alpha,
        'gam': gam
    }

def find_omega_invariant_set(A, B, verbosity=0):
    eig = np.linalg.eig(A)[0]
    
    # we use fmin to solve a line search problem in alpha for minimum gamma
    if verbosity > 0:
        print('line search')
    
    # we perform a line search over alpha to find the largest convergence rate possible
    alpha_1 = -np.real(np.max(eig)) # smallest magnitude value from eig-value, and range has to be positive
    alpha_opt = scipy.optimize.fminbound(lambda alpha: solve_lmi(alpha, A, B, verbosity=verbosity)['cost'], x1=1e-5, x2=alpha_1, disp=True if verbosity > 0 else False)
    
    # if the alpha optimization fail, pick a fixed value for alpha.
    sol = solve_lmi(alpha_opt, A, B)
    prob = sol['prob']
    if prob.status == 'optimal':
        P = prob.variables['P'].value
        mu1 =  prob.variables['mu_1'].value
        if verbosity > 0:
            print(sol)
    else:
        raise RuntimeError('Optimization failed')
        
    return sol